In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import torchvision
import torchvision.transforms as transforms
from torchvision.datasets import ImageFolder
import timm
import sys



print('System Version:', sys.version)
print('PyTorch version', torch.__version__)
print('Torchvision version', torchvision.__version__)

System Version: 3.10.12 | packaged by conda-forge | (main, Jun 23 2023, 22:40:32) [GCC 12.3.0]
PyTorch version 2.0.0
Torchvision version 0.15.1


Began by importing the torch libraries as well as timm. timm has prepackaged tools that will be used for image classification in the pytorch model. After this, I created a pytorch dataset by inheriting from the Dataset class. I added two methods, len and getitem that I test in the two cells following the class. Additionally, I add a transform to resize the images in my data directory to a standard size of 128,128x3 since the model requires a standard image size. 

In [2]:
#creating an iteratable object that we can loop over to train data
class VirusDataset(Dataset):
    def __init__(self, data_dir, transform = None):
        #ImageFolder class from torchvision package and provide it with the data directory as well as the transform it will be given
        self.data = ImageFolder(data_dir, transform = transform)

    #pytorch method that tells data loader how many examples are in a dataset
    def __len__(self):
        return len(self.data)

    #pytorch method that takes an index in dataset and returns one item    
    def __getitem__(self, idx):
        return self.data[idx]
    
    #I could add another method for classes that returns data classes from ImageFolder if I wanted to visualize the model results
    #This would retrieve class names i.e CoronaVirus so I could display them on an axis 

In [3]:
#Testing the len method in the class VirusDataset
dataset = VirusDataset(data_dir = '/kaggle/input/virusmodeldataset/Data_Sources/Dataset_images/train')
len(dataset)

20

In [4]:
#Testing the getitem method in the class VirusDataset
image, label = dataset[16]
print(label)


3


In [5]:
data_dir = '/kaggle/input/virusmodeldataset/Data_Sources/Dataset_images/train'
#create disctionary target_to_class that associates given number with proper label
#in the cell above, dataset[16] gives 3, which would be in the Rhinovirus
#This makes sense because each training dataset has 5 images, so 0 = 0-4, 1 = 5-9...etc
target_to_class = {v: k for k, v in ImageFolder(data_dir).class_to_idx.items()}
print(target_to_class)

{0: 'Coronavirus', 1: 'Herpesvirus', 2: 'Influenzavirus', 3: 'Rhinovirus'}


In [6]:
#convert image to a standard size of 128x128x3 (3 = RGB channels) and convert to pytorch tensor
transform = transforms.Compose([transforms.Resize((128,128)), transforms.ToTensor()])
data_dir = '/kaggle/input/virusmodeldataset/Data_Sources/Dataset_images/train'
dataset = VirusDataset(data_dir, transform)
#check size for a random image
image, label = dataset[15]
image.shape

torch.Size([3, 128, 128])

Next I wrap the created dataset with a pytorch dataloader that will process the images in the dataset. This step will batch the data when it is given to the actual model, especially helpful for large datasets (not the case with my test data)

In [7]:
#call pytorch dataloader and provide dataset
#add a shuffle since objective is training data, will not be shuffling for test or validation
dataloader = DataLoader(dataset, batch_size = 10, shuffle = True)

In [8]:
#test dataloader to see images and labels for a batch
for images, labels in dataloader:
    break
labels, images


(tensor([2, 1, 2, 3, 3, 0, 3, 0, 0, 1]),
 tensor([[[[0.0000, 0.0000, 0.0039,  ..., 0.0353, 0.0392, 0.0353],
           [0.0157, 0.0118, 0.0157,  ..., 0.0431, 0.0471, 0.0431],
           [0.0353, 0.0275, 0.0235,  ..., 0.0549, 0.0549, 0.0510],
           ...,
           [0.0392, 0.0588, 0.0667,  ..., 0.0314, 0.0078, 0.0157],
           [0.0353, 0.0588, 0.0627,  ..., 0.0157, 0.0314, 0.0157],
           [0.0235, 0.0510, 0.0549,  ..., 0.0235, 0.0196, 0.0157]],
 
          [[0.1020, 0.1098, 0.1412,  ..., 0.3725, 0.3529, 0.3294],
           [0.1216, 0.1255, 0.1529,  ..., 0.3725, 0.3529, 0.3294],
           [0.1373, 0.1373, 0.1608,  ..., 0.3765, 0.3569, 0.3333],
           ...,
           [0.4157, 0.4706, 0.5137,  ..., 0.0980, 0.0706, 0.0824],
           [0.4118, 0.4706, 0.5098,  ..., 0.0824, 0.0980, 0.0824],
           [0.4000, 0.4627, 0.5020,  ..., 0.0863, 0.0863, 0.0784]],
 
          [[0.1686, 0.1725, 0.1961,  ..., 0.4510, 0.4353, 0.4196],
           [0.1882, 0.1882, 0.2078,  ..., 0.4549, 

For the pytorch model, timm is implemented and this model uses the efficientnet_b0 model since it is a relatively fast training model. Setting pretrained to true means the model weights which are the values within the model's layers that are adjusted during training, have been adjusted from the imagenet dataset. The default feature size of the efficientnet model which is 1280 needs to match the class size of the model im creating. Additionally, the efficientnet model has an extra layer that needs to be removed as a result of needing to match the feature size with num_classes.

In [9]:
#create class that imports from the neural network module of pytorch
class VirusClassifier(nn.Module):
    def __init__(self, num_classes = 4):
        #Initialize object with parent class using super()
        super(VirusClassifier, self).__init__()
        # Defining the model
        self.base_model = timm.create_model('efficientnet_b0', pretrained=True)
        self.features = nn.Sequential(*list(self.base_model.children())[:-1])

        enet_out_size = 1280
        # Make a classifier to adjust enet_out size to match class size
        self.classifier = nn.Sequential(
            nn.Flatten(),
            nn.Linear(enet_out_size, num_classes)
        )
    
    def forward(self, x):
        # Connect parts of the defined model and returning output
        x = self.features(x)
        output = self.classifier(x)
        return output
        

In [10]:
#model is object/instance of VirusClassifier
#test and view model details
model = VirusClassifier(num_classes = 4)
print(str(model)[:1000])

VirusClassifier(
  (base_model): EfficientNet(
    (conv_stem): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
    (bn1): BatchNormAct2d(
      32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True
      (drop): Identity()
      (act): SiLU(inplace=True)
    )
    (blocks): Sequential(
      (0): Sequential(
        (0): DepthwiseSeparableConv(
          (conv_dw): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
          (bn1): BatchNormAct2d(
            32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True
            (drop): Identity()
            (act): SiLU(inplace=True)
          )
          (se): SqueezeExcite(
            (conv_reduce): Conv2d(32, 8, kernel_size=(1, 1), stride=(1, 1))
            (act1): SiLU(inplace=True)
            (conv_expand): Conv2d(8, 32, kernel_size=(1, 1), stride=(1, 1))
            (gate): Sigmoid()
          )
          (conv_pw): Conv2d(32, 16, kernel_

In [11]:
#testing forward model by calling model with ex. images to see if batch size and class size are accurate
test_out = model(images)
test_out.shape

torch.Size([10, 4])

Before beginning the training loop, the datasets are set up so datasets are made from the path and dataloaders are made for each set. 

In [12]:
transform = transforms.Compose([
    transforms.Resize((128, 128)),
    transforms.ToTensor(),
])

train_folder = '/kaggle/input/virusmodeldataset/Data_Sources/Dataset_images/train'
valid_folder = '/kaggle/input/virusmodeldataset/Data_Sources/Dataset_images/valid'
test_folder = '/kaggle/input/virusmodeldataset/Data_Sources/Dataset_images/test'

train_dataset = VirusDataset(train_folder, transform=transform)
valid_dataset = VirusDataset(valid_folder, transform=transform)
test_dataset = VirusDataset(test_folder, transform=transform)

#only want shuffling when model is training, validation and test are there just for that, to validate and test the training
train_loader = DataLoader(train_dataset, batch_size=10, shuffle=True)
valid_loader = DataLoader(valid_dataset, batch_size=10, shuffle=False)
test_loader = DataLoader(valid_dataset, batch_size=10, shuffle=False)

In [13]:
#set to train with 5 runs through entire dataset
num_epochs = 5
train_losses, valid_losses = [], []

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

model = VirusClassifier(num_classes = 4)
model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr = 0.001)

for epoch in range(num_epochs):
    model.train()
    #running_loss will accumulate total loss across batches per epoch
    running_loss = 0.0
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item() + images.size(0)
    #after one loop of epoch complete, store training loss
    #training loss is avg loss per data point in data set per epoch
    train_loss = running_loss / len(train_loader.dataset)
    train_losses.append(train_loss)
    
    #validation, change model from training to validation
    model.eval()
    running_loss = 0.0
    with torch.no_grad():
        for images, labels in valid_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)
            running_loss += loss.item() * images.size(0)
    valid_loss = running_loss / len(valid_loader.dataset)
    valid_losses.append(valid_loss)
    
    print(f"Epoch {epoch+1}/{num_epochs} - Train loss: {train_loss}, Validation loss: {valid_loss}")
    

Epoch 1/5 - Train loss: 1.1287650406360625, Validation loss: 1.1345195770263672
Epoch 2/5 - Train loss: 1.0436465844511986, Validation loss: 1.010746955871582
Epoch 3/5 - Train loss: 1.013534390926361, Validation loss: 0.9933407306671143
Epoch 4/5 - Train loss: 1.0049243174493312, Validation loss: 0.9730596542358398
Epoch 5/5 - Train loss: 1.0018904163502156, Validation loss: 0.9559248685836792


As the model trains through the epochs, the training loss and validation loss is shown per epoch using the cross entropy loss criterion, which measures difference between predicted and actual probabilities. I am using cross entropy loss since my data has 4 classes and thus is a multi-class classification. 

The loss values are very high, and this could be due to many reasons. Primarily, my dataset was a test dataset that I created myself and is extremely small. Additionally, the images are very noisy. The model may be doing something such as predicting incorrectly but with high confidence, leading to a higher loss. 

In [21]:
from sklearn.metrics import f1_score
import numpy as np

#In order to measure model accuracy, evaluated pre
# Evaluation mode
model.eval()

#Variables to compute f1 score using f1_score function from sklearn.metrics
predicted_labels = []
true_labels = []

with torch.no_grad():
    for images, labels in valid_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs, 1)
        predicted_labels.extend(predicted.cpu().numpy())
        true_labels.extend(labels.cpu().numpy())


f1 = f1_score(true_labels, predicted_labels, average='weighted')

print(f"F1 Score: {f1}")

F1 Score: 0.5666666666666667


The F1 score supports my prediction of the model having low accuracy due to data sample size and image quality. For future projects, I can implement a very similar paradigm that is common through a lot of pytorch models but use a dataset with less noisy images and a much larger dataset for a higher f1 score - a higher model accuracy. 